In [34]:
#%pip install python-Levenshtein
#%pip install keras-ocr
%pip uninstall -y tensorflow tensorflow-cpu protobuf
%pip cache purge
%pip install tensorflow-cpu==2.10 protobuf==3.20.* --no-deps
%pip install tensorflow-cpu


Found existing installation: tensorflow_cpu 2.19.0Note: you may need to restart the kernel to use updated packages.

Uninstalling tensorflow_cpu-2.19.0:
  Successfully uninstalled tensorflow_cpu-2.19.0
Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4


Files removed: 9
Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 904.0/904.0 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 262.7/262.7 MB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 7.2 MB/s eta 0:00:00
     -------------------------------------- 895.7/895.7 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.9.1
    Uninstalling keras-3.9.1:
      Successfully uninstalled keras-3.9.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstal


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
%pip show protobuf
%pip show tensorflow
import tensorflow as tf
print(tf.__version__)

Name: protobuf
Version: 3.19.6
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: 3-Clause BSD License
Location: c:\users\adam4\appdata\local\programs\python\python310\lib\site-packages
Requires: 
Required-by: tensorboard, tensorflow_intel
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [14]:
import os
from difflib import SequenceMatcher
import Levenshtein

# Directories
expected_output_directory = 'cleaned/000'
actual_output_directory = 'outputs/py_cleaned'

# METHOD 1:
# Similarity percentage using Levenshtein distance (edit distance), more robust for OCR
# Levenshtein distance outputs how similar, not dis-similar
def calculate_similarity_lev(text1, text2):
    distance = Levenshtein.distance(text1, text2)
    max_length = max(len(text1), len(text2))
    if max_length == 0:  # Avoids dividing by zero
        return 100.0
    return (1 - distance / max_length) * 100

# METHOD 2:
def calculate_similarity(text1, text2): 
    return SequenceMatcher(None, text1, text2).ratio() * 100


def openCalculate(path):
    if os.path.exists(path):
                with open(path, 'r') as f:
                    result = f.read()
    return result

In [15]:
def VERSION1pyTesseractProcessWrittenOutput(output):
    resultText = []
    resultWritten = []
    Text = True

    with open(output, 'r') as file:
        lines = file.readlines()
        for line in lines[2:]: #first two lines in pytesseract are not useful
            if Text:
                if line.strip() == "":  # Blank newline indicates the end of Written Text #since pytesseract is better at text dectection
                    Text = False
                    continue
                resultText.append(line.strip())
            elif Text == False:
                if line.strip().startswith("Name:"):  # Stop when "Name:" is found (end of file) does need to end at "" since might read text werid
                    break
                resultWritten.append(line.strip())

    return resultText, resultWritten

#def

def kerasOCRProcessWrittenOutput(output):
    resultText = []
    resultWritten = []
    #NEED TO FINISH THIS FUNCTION
    return resultText, resultWritten

def easyOCRProcessWrittenOutput(output):
    resultText = []
    resultWritten = []
    #NEED TO FINISH THIS FUNCTION
    return resultText, resultWritten

def groundTruthProcessWrittenOutput(output):
    resultText = []
    for line in output.splitlines():
        line = line.strip()
        if line:
            resultText.append(line)
    return resultText

In [16]:
def loadDataAll(groundTruth, pyTesseractOutputTyped, pyTesseractOutputWritten, kerasOCROutputTyped, kerasOCROutputWritten, easyOCROutputTyped, easyOCROutputWritten):
    count = 0
    pyTesseractTypedScore = 0
    pyTesseractWrittenScore = 0
    kerasOCRTypedScore = 0
    kerasOCRWrittenScore = 0
    easyOCRTypedScore = 0
    easyOCRWrittenScore = 0

    for filename in os.listdir(groundTruth):
        if filename.endswith('.txt'):
            groundTruthPath = os.path.join(groundTruth, filename)
            pyTesseractOutputPathWritten = os.path.join(pyTesseractOutputTyped, filename)
            pyTesseractOutputPathTyped = os.path.join(pyTesseractOutputWritten, filename)
            kerasOCROutputPathTyped = os.path.join(kerasOCROutputTyped, filename)
            kerasOCROutputPathWritten = os.path.join(kerasOCROutputWritten, filename)
            easyOCROutputPathTyped = os.path.join(easyOCROutputTyped, filename)
            easyOCROutputPathWritten = os.path.join(easyOCROutputWritten, filename)
    
            groundTruth = openCalculate(groundTruthPath)
            pyTesseractOutputTyped = openCalculate(pyTesseractOutputPathTyped)
            pyTesseractOutputWritten = openCalculate(pyTesseractOutputPathWritten)
            kerasOCROutputTyped = openCalculate(kerasOCROutputPathTyped)
            kerasOCROutputWritten = openCalculate(kerasOCROutputPathWritten)
            easyOCROutputTyped = openCalculate(easyOCROutputPathTyped)
            easyOCROutputWritten = openCalculate(easyOCROutputPathWritten)

            pyTesseractTypedScoreLevScore = calculate_similarity_lev(pyTesseractOutputTyped, groundTruth)
            pyTesseractWrittenScoreLevScore = calculate_similarity_lev(pyTesseractOutputWritten, groundTruth)
            kerasOCRTypedScoreLevScore = calculate_similarity_lev(kerasOCROutputTyped, groundTruth)
            kerasOCRWrittenScoreLevScore = calculate_similarity_lev(kerasOCROutputWritten, groundTruth)
            easyOCRTypedScoreLevScore = calculate_similarity_lev(easyOCROutputTyped, groundTruth)
            easyOCRWrittenScoreLevScore = calculate_similarity_lev(easyOCROutputWritten, groundTruth)
            
            #similarity = calculate_similarity(expected_text, actual_text)
    
            print(f"File: {filename}")
            print(f"Similarity Percentage from Levenshtein with PyTesseract Typed: {pyTesseractTypedScore:.2f}%")
            print(f"Similarity Percentage from Levenshtein with PyTesseract Written: {pyTesseractWrittenScore:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Keras OCR Typed: {kerasOCRTypedScore:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Keras OCR Written: {kerasOCRWrittenScore:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Easy OCR Typed: {easyOCRTypedScore:.2f}%")
            count += 1
            pyTesseractTypedScore += pyTesseractTypedScoreLevScore
            pyTesseractWrittenScore += pyTesseractWrittenScoreLevScore
            kerasOCRTypedScore += kerasOCRTypedScoreLevScore
            kerasOCRWrittenScore += kerasOCRWrittenScoreLevScore
            easyOCRTypedScore += easyOCRTypedScoreLevScore
            easyOCRWrittenScore += easyOCRWrittenScoreLevScore
        else:
                print(f"Expected output not found for {filename}")
        print("---------------------------------------------------")

        result = {
            "pytesseract Typed": pyTesseractTypedScore / count,
            "pytesseract Written": pyTesseractWrittenScore / count,
            "kerasOCR Typed": kerasOCRTypedScore / count,
            "kerasOCR Written": kerasOCRWrittenScore / count,
            "easyOCR Typed": easyOCRTypedScore / count,
            "easyOCR Written": easyOCRWrittenScore / count
        }
        return result

# PyTesseract

In [17]:
import pytesseract
from PIL import Image
import os

#Tesseract executable
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"

image_directory = 'cleaned/000-cropped'

#expected output directory as text file
#output_directory = 'cleaned/000'
#file 1 in one directory will be file 1 in another directory

for filename in os.listdir(image_directory):

    if filename.endswith('.png'):
        image_path = os.path.join(image_directory, filename)

        # Open using PIL
        img = Image.open(image_path)

        # Output file format
        #output_file ="outputs/pytesseract/" + os.path.splitext(filename)[0] + '.txt'
        output_file ="outputs/py_cleaned/" + os.path.splitext(filename)[0] + '.txt'

        # Run Tesseract and writes into output file
        with open(output_file, 'w') as f:
            text = pytesseract.image_to_string(img)
            f.write(text)

        print(f"Text extracted from {filename} and saved to {output_file}")


Text extracted from a01-000.png and saved to outputs/py_cleaned/a01-000.txt
Text extracted from a01-003.png and saved to outputs/py_cleaned/a01-003.txt
Text extracted from a01-007.png and saved to outputs/py_cleaned/a01-007.txt
Text extracted from a01-011.png and saved to outputs/py_cleaned/a01-011.txt
Text extracted from a01-014.png and saved to outputs/py_cleaned/a01-014.txt
Text extracted from a01-020.png and saved to outputs/py_cleaned/a01-020.txt
Text extracted from a01-026.png and saved to outputs/py_cleaned/a01-026.txt
Text extracted from a01-030.png and saved to outputs/py_cleaned/a01-030.txt
Text extracted from a01-043.png and saved to outputs/py_cleaned/a01-043.txt
Text extracted from a01-049.png and saved to outputs/py_cleaned/a01-049.txt
Text extracted from a01-049x.png and saved to outputs/py_cleaned/a01-049x.txt
Text extracted from a01-053.png and saved to outputs/py_cleaned/a01-053.txt
Text extracted from a01-058.png and saved to outputs/py_cleaned/a01-058.txt
Text extra

# Keras-OCR

In [21]:
import matplotlib.pyplot as plt

import keras_ocr

# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

# Get a set of three example images
images = [
    keras_ocr.tools.read(url) for url in [
        'https://upload.wikimedia.org/wikipedia/commons/b/bd/Army_Reserves_Recruitment_Banner_MOD_45156284.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/e/e8/FseeG2QeLXo.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/b/b4/EUBanana-500x112.jpg'
    ]
]

# Each list of predictions in prediction_groups is a list of
# (word, box) tuples.
prediction_groups = pipeline.recognize(images)

# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for ax, image, predictions in zip(axs, images, prediction_groups):
    keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)

ImportError: Traceback (most recent call last):
  File "c:\Users\adam4\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

# EasyOCR

In [ ]:
import easyocr
import os

reader = easyocr.Reader(['en'])

folder_path = 'cleaned/000-cropped'

for filename in os.listdir(folder_path):

    if filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)

        output = reader.readtext(image_path, detail=0)

        output_file ="outputs/easy-ocr-cleaned/" + os.path.splitext(filename)[0] + '.txt'

        with open(output_file, 'w') as f:
            for line in output:
                f.write(line + '\n')  

        print(f"Text for {filename} written to {output_file}")


Text for a01-000.png written to outputs/easy-ocr-cleaned/a01-000.txt
Text for a01-003.png written to outputs/easy-ocr-cleaned/a01-003.txt
Text for a01-007.png written to outputs/easy-ocr-cleaned/a01-007.txt
Text for a01-011.png written to outputs/easy-ocr-cleaned/a01-011.txt
Text for a01-014.png written to outputs/easy-ocr-cleaned/a01-014.txt
Text for a01-020.png written to outputs/easy-ocr-cleaned/a01-020.txt
Text for a01-026.png written to outputs/easy-ocr-cleaned/a01-026.txt
Text for a01-030.png written to outputs/easy-ocr-cleaned/a01-030.txt
Text for a01-043.png written to outputs/easy-ocr-cleaned/a01-043.txt
Text for a01-049.png written to outputs/easy-ocr-cleaned/a01-049.txt
Text for a01-049x.png written to outputs/easy-ocr-cleaned/a01-049x.txt
Text for a01-053.png written to outputs/easy-ocr-cleaned/a01-053.txt
Text for a01-058.png written to outputs/easy-ocr-cleaned/a01-058.txt
Text for a01-063.png written to outputs/easy-ocr-cleaned/a01-063.txt
Text for a01-068.png written to 

# Running Comparison

In [ ]:
# Load the data and calculate similarity scores
result = loadDataAll(expected_output_directory, actual_output_directory, actual_output_directory, actual_output_directory, actual_output_directory, actual_output_directory, actual_output_directory)
print("---------------------------------------------------")
print("Final Results")
print("---------------------------------------------------")
print(f"Similarity Percentage from Levenshtein with PyTesseract Typed: {result['pytesseract Typed']:.2f}%")
print(f"Similarity Percentage from Levenshtein with PyTesseract Written: {result['pytesseract Written']:.2f}%")
print(f"Similarity Percentage from Levenshtein with Keras OCR Typed: {result['kerasOCR Typed']:.2f}%")
print(f"Similarity Percentage from Levenshtein with Keras OCR Written: {result['kerasOCR Written']:.2f}%")
print(f"Similarity Percentage from Levenshtein with Easy OCR Typed: {result['easyOCR Typed']:.2f}%")
print(f"Similarity Percentage from Levenshtein with Easy OCR Written: {result['easyOCR Written']:.2f}%")
print("---------------------------------------------------")
print("End of Program")